## 양방향 LSTM과 어텐션 메커니즘(BiLSTM with Attention mechanism)

####  IMDB 리뷰 데이터 전처리하기

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences